In [17]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import os
import warnings
import joblib
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv
import torch
from torchtuples.practical import MLPVanilla
from sklearn.linear_model import LassoCV
from torchtuples import optim as ttoptim


In [18]:

# Suppress all warnings
warnings.filterwarnings('ignore')


In [19]:

model_used = 'CoxPH'
selection_method = 'lasso'

Light_GBM_global = [
    "flair_t1ce_t2",
    "flair_t1ce_t2",
    "flair",
    "flair_t1ce",
    "flair_t1ce"
]
Light_GBM_local = [
    "flair_t1ce_t2",
    "flair_t1ce",
    "flair",
    "t2",
    "flair_t1ce",
]

In [ ]:
train_path = '../dataset/MICCAI_BraTS2020_TrainingData/survival_info.csv'
val_path = '../dataset/MICCAI_BraTS2020_ValidationData/survival_evaluation.csv'

In [ ]:
def get_events(csv_file_path):
    df = pd.read_csv(csv_file_path)
    return df['Events'].values

In [20]:

def ensure_directory_exists(filepath):
    os.makedirs(os.path.dirname(filepath), exist_ok=True)


In [21]:
def load_radiomic_features(target_directory, file_name):
    file_path = os.path.join(target_directory, file_name)
    array = np.load(file_path)
    print(f"Array loaded from '{file_path}'")
    return array


In [22]:
def save_model_and_parameters(model, modality_used, mask):
    model_dir = f"./models/{selection_method}_feature_selection/{model_used}/{modality_used}/"
    ensure_directory_exists(model_dir)
    model_file = os.path.join(model_dir, 'model.pth')
    mask_file = os.path.join(model_dir, f'{selection_method}_mask.npy')
    
    torch.save(model.state_dict(), model_file)
    np.save(mask_file, mask)
    
    print(f"Model and {selection_method} mask saved successfully for modality {modality_used}.")


In [23]:
def load_and_combine_features(modality_keys, dataset_type):
    combined_features = []
    for modality in modality_keys:
        features = np.load(f'../local_spatial_Framework/features/{modality}/{dataset_type}/{dataset_type}_backbone_outputs.npy')
        combined_features.append(features)
    return np.concatenate(combined_features, axis=1)


In [24]:


def make_csv(y_pred_validation, modality_used):
    df = pd.read_csv('../dataset/MICCAI_BraTS2020_ValidationData/survival_evaluation.csv')
    validation_ids = df['BraTS20ID'].values
    filename = f"../radiomics_local_global_predictions/{selection_method}_feature_selection/{model_used}/{modality_used}_{model_used}.csv"

    ensure_directory_exists(filename)

    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["ID", "Days"])
        for id, day in zip(validation_ids, y_pred_validation):
            writer.writerow([id, day])

    print(f"CSV file '{filename}' created successfully.")


In [25]:

def load_features(modality_used):
    base_dir = os.path.join('../Global_extracted_features', modality_used)
    train_features = np.load(os.path.join(base_dir, 'train_features.npy'))
    validate_features = np.load(os.path.join(base_dir, 'validate_features.npy'))
    train_labels = np.load(os.path.join(base_dir, 'train_labels.npy'))
    return train_features, validate_features, train_labels

In [26]:
def train_model(train_features, validate_features, train_labels, modality_used):
    scaler = StandardScaler()
    train_features_scaled = scaler.fit_transform(train_features)
    validate_features_scaled = scaler.transform(validate_features)
    
    print(f"Size of features before Lasso: {train_features_scaled.shape}")

    lasso = LassoCV(cv=5, random_state=42, max_iter=100, alphas=np.logspace(-4, -0.5, 30)).fit(train_features_scaled, train_labels)
    
    mask = lasso.coef_ != 0
    train_features_selected = train_features_scaled[:, mask]
    validate_features_selected = validate_features_scaled[:, mask]

    if train_features_selected.shape[1] == 0:
        print(f"No features selected for modality {modality_used}. Skipping this combination.")
        return

    print(f"Size of features after Lasso: {train_features_selected.shape}")

    # Define PyCox model
    net = MLPVanilla(train_features_selected.shape[1], [32, 32], 1, torch.nn.ReLU)
    model = CoxPH(net, ttoptim.Adam)
    
    # Prepare the dataset
    train_data = (train_features_selected, train_labels)
    val_data = (validate_features_selected, validate_features[:, 0])  # Assuming validate_features contains survival time and event
    
    # Train the model
    model.fit(train_data, epochs=100, batch_size=256, val_data=val_data, val_batch_size=256)
    
    # Make predictions
    surv = model.predict_surv_df(validate_features_selected)
    
    # Evaluate predictions
    ev = EvalSurv(surv, validate_features[:, 0], validate_features[:, 1], censor_surv='km')
    c_index = ev.concordance_td('antolini')
    
    # Save results
    make_csv(surv.median().values, modality_used)
    save_model_and_parameters(model, modality_used, mask)

In [27]:

radiomic_train_features = load_radiomic_features("../radiomics features/all", "radiomics_train.npy")
radiomic_validation_features = load_radiomic_features("../radiomics features/all", "radiomics_validate.npy")

print("radiomic_train_features", radiomic_train_features.shape)
print("radiomic_validation_features", radiomic_validation_features.shape)


Array loaded from '../radiomics features/all/radiomics_train.npy'
Array loaded from '../radiomics features/all/radiomics_validate.npy'
radiomic_train_features (235, 400)
radiomic_validation_features (28, 400)


In [28]:


for modality_used_global, modality_used_local in zip(Light_GBM_global, Light_GBM_local):
    modality_key_local = modality_used_local.split("_")
    modality_keys_list_global_features = modality_used_global.split("_")
    print(f"\nLoading and combining features... \n local-{modality_used_local}\n global-{modality_used_global}")

    local_train_features = load_and_combine_features(modality_key_local, 'train')
    local_validation_features = load_and_combine_features(modality_key_local, 'validation')
    
    global_train_features, global_validate_features, train_labels = load_features(modality_used_global)
    
    print("global_train_features", global_train_features.shape, "local_train_features", local_train_features.shape)
    print("global_validate_features", global_validate_features.shape, "local_validation_features", local_validation_features.shape)

    local_global_training_features = np.concatenate((global_train_features, local_train_features), axis=1)
    local_global_validation_features = np.concatenate((global_validate_features, local_validation_features), axis=1)

    train_labels = np.delete(train_labels, 98, axis=0)
    local_global_training_features = np.delete(local_global_training_features, 98, axis=0)
    local_global_validation_features = np.delete(local_global_validation_features, 27, axis=0)

    training_all_features = np.concatenate((local_global_training_features, radiomic_train_features), axis=1)
    validation_all_features = np.concatenate((local_global_validation_features, radiomic_validation_features), axis=1)

    print("combining all")
    print("radiomics_local_global_training", training_all_features.shape)
    print("radiomics_local_global_validation", validation_all_features.shape)

    modality_used = 'global_' + modality_used_global + '___local_' + modality_used_local
    
    train_model(training_all_features, validation_all_features, train_labels, modality_used)



Loading and combining features... 
 local-flair_t1ce_t2
 global-flair_t1ce_t2
global_train_features (236, 744) local_train_features (236, 192)
global_validate_features (29, 744) local_validation_features (29, 192)
combining all
radiomics_local_global_training (235, 1336)
radiomics_local_global_validation (28, 1336)
Size of features before Lasso: (235, 1336)
Size of features after Lasso: (235, 670)


TypeError: fit() missing 1 required positional argument: 'target'